In [1]:
import sys
sys.path.insert(1, r'C:\Users\Donald\PycharmProjects\monkey_nuc_segment\stardist\workflow')
from functions import TileSetScorer, ScoringSubroutine
from __future__ import print_function, unicode_literals, absolute_import, division
import sys
import numpy as np
import matplotlib
import json
matplotlib.rcParams["image.interpolation"] = 'nearest'
import matplotlib.pyplot as plt
import os
from PIL import Image
import tensorflow as tf
from glob import glob
from tqdm import tqdm
from tifffile import imread
import struct
import pandas as pd
import cv2
from csbdeep.utils import Path, normalize
from glob import glob

from stardist import fill_label_holes, random_label_cmap, calculate_extents, gputools_available
from stardist.models import Config2D, StarDist2D, StarDistData2D

In [2]:
def load_model(model_path: str) -> StarDist2D:
    # Load StarDist model weights, configurations, and thresholds
    with open(model_path + '\\config.json', 'r') as f:
        config = json.load(f)
    with open(model_path + '\\thresholds.json', 'r') as f:
        thresh = json.load(f)
    model = StarDist2D(config=Config2D(**config), basedir=model_path, name='offshoot_model')
    model.thresholds = thresh
    print('Overriding defaults:', model.thresholds, '\n')
    model.load_weights(model_path + '\\weights_best.h5')
    return model

In [3]:
model_pths = r'\\10.99.68.178\andreex\data\Stardist\12_12_hyperparameter_models'
pths = os.listdir(model_pths)
pths = ["".join([model_pths, '\\', pth]) for pth in pths if 'monkey' in pth]

In [4]:
models = [load_model(pth) for pth in pths]

base_model.py (149): output path for model already exists, files may be overwritten: \\10.99.68.178\andreex\data\Stardist\12_12_hyperparameter_models\monkey_12_12_2023_lr_0.0005_epochs_100_pt_10_gaus_ratio_0\offshoot_model


Using default values: prob_thresh=0.5, nms_thresh=0.4.
Overriding defaults: Thresholds(prob=0.7193147122688542, nms=0.3) 


base_model.py (149): output path for model already exists, files may be overwritten: \\10.99.68.178\andreex\data\Stardist\12_12_hyperparameter_models\monkey_12_12_2023_lr_0.0005_epochs_100_pt_10_gaus_ratio_0.1\offshoot_model


Using default values: prob_thresh=0.5, nms_thresh=0.4.
Overriding defaults: Thresholds(prob=0.7175685959330165, nms=0.3) 


base_model.py (149): output path for model already exists, files may be overwritten: \\10.99.68.178\andreex\data\Stardist\12_12_hyperparameter_models\monkey_12_12_2023_lr_0.0005_epochs_100_pt_20_gaus_ratio_0\offshoot_model


...

Using default values: prob_thresh=0.5, nms_thresh=0.4.
Overriding defaults: Thresholds(prob=0.09494988367404089, nms=0.3) 


base_model.py (149): output path for model already exists, files may be overwritten: \\10.99.68.178\andreex\data\Stardist\12_12_hyperparameter_models\monkey_12_12_2023_lr_0.005_epochs_400_pt_40_gaus_ratio_0\offshoot_model


Using default values: prob_thresh=0.5, nms_thresh=0.4.
Overriding defaults: Thresholds(prob=0.5706334814401934, nms=0.3) 


base_model.py (149): output path for model already exists, files may be overwritten: \\10.99.68.178\andreex\data\Stardist\12_12_hyperparameter_models\monkey_12_12_2023_lr_0.005_epochs_400_pt_40_gaus_ratio_0.1\offshoot_model


Using default values: prob_thresh=0.5, nms_thresh=0.4.
Overriding defaults: Thresholds(prob=0.12045020965240208, nms=0.3) 


In [5]:
# load tile images and masks
pthHE = r'\\10.99.68.178\andreex\data\Stardist\Testing\monkey\tiles'
pthmasks = r'\\10.99.68.178\andreex\data\Stardist\Testing\monkey\qupath\ground_truth\masks'

HE_tiles_pths = sorted(glob(os.path.join(pthHE,'*.tif')))
mask_tiles_pths = sorted(glob(os.path.join(pthmasks,'*.tif')))

HE_tiles = list(map(imread,HE_tiles_pths))
mask_tiles = list(map(imread,mask_tiles_pths))

HE_tiles = [x/255 for x in HE_tiles]  # just divide by 255 to get into (0,1) range
mask_tiles = [fill_label_holes(y) for y in mask_tiles]  # fills holes in annotations if there are any

In [6]:
def get_predictions(model):
    mask_val_pred = [model.predict_instances(x, n_tiles=model._guess_n_tiles(x), show_tile_progress=False)[0]
              for x in tqdm(HE_tiles)]

    return mask_val_pred

In [7]:
def get_f1(tp, fp, fn):
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    avg_precision = tp / (tp + fp + fn)
    f1 = 2 * precision * recall / (precision + recall)
    return precision, recall, avg_precision, f1

In [8]:
def find_first_unique_indices(lst):
    unique_indices = {}
    result = []

    for i, value in enumerate(lst):
        if value not in unique_indices:
            unique_indices[value] = i

    for value in lst:
        result.append(unique_indices[value])

    return result

In [9]:
def get_organ_f1(result):
    names = result['Image']

    names = [name.split(".")[0][21:] for name in names]

    for i in range(len(names)):
        #if len(names[i]) > 6:
        names[i] = names[i][:2]

    result_list = find_first_unique_indices(names)

    f1_scores = result['F1 Score']

    seen = []
    names = []

    raw_vals = result[['Image', 'TP', 'FP', 'FN', 'Avg Precision']]

    TPs = [0 for _ in range(len(raw_vals))]
    FPs = [0 for _ in range(len(raw_vals))]
    FNs = [0 for _ in range(len(raw_vals))]
    APs = [0 for _ in range(len(raw_vals))]
    PQs = [0 for _ in range(len(raw_vals))]

    for i in range(len(raw_vals)):
        row = raw_vals.iloc[i]
        num = result_list[i]
        if num not in seen:
            names.append(raw_vals['Image'][i])
            seen.append(num)

        TPs[num] += row['TP']
        FPs[num] += row['FP']
        FNs[num] += row['FN']
        APs[num] += row['Avg Precision']
        PQs[num] += row['pan_qual']

    organ_names = []
    for i in range(len(names)):
        nm = names[i][21:-4]
        if nm[-1] == '1':
            nm = nm[:-1]
        organ_names.append(nm)

    TPs = [val for val in TPs if val != 0]
    FPs = [val for val in FPs if val != 0]
    FNs = [val for val in FNs if val != 0]
    APs = [val for val in APs if val != 0]
    PQs = [val for val in APs if val != 0]

    if [len(organ_names), len(TPs), len(FPs), len(FNs)] != [len(organ_names), len(organ_names), len(organ_names), len(organ_names)]:
        print('skipped')
        return 0

    new_df = pd.DataFrame({'Organ': organ_names, 'TP': TPs, 'FP': FPs, 'FN': FNs, 'AP': APs})

    precisions = []
    recalls = []
    f1s = []
    APs = []

    for i in range(len(new_df)):
        row = new_df.iloc[i]

        tp = row['TP']
        fp = row['FP']
        fn = row['FN']

        precision, recall, avg_precision, f1 = get_f1(tp, fp, fn)
        precisions.append(precision)
        recalls.append(recall)
        f1s.append(f1)
        APs.append(avg_precision)

    new_df['Recall'] = recalls
    new_df['Precision'] = precisions
    new_df['F1'] = f1s
    new_df['AP'] = APs
    new_df['PQ'] = PQs

    return new_df

In [10]:
tau = 0.7
nms = [os.path.basename(file) for file in HE_tiles_pths]
predictions = [get_predictions(model) for model in models]

100%|██████████| 36/36 [00:05<00:00,  6.65it/s]


In [11]:
scores = []

for model_prediction in predictions:
    score = TileSetScorer(nms, mask_tiles, model_prediction, [tau])
    scores.append(score)

In [14]:
scores[0].score_set()

,Image,Tau,IoU,TP,FP,FN,Precision,Recall,Avg Precision,F1 Score,Seg Quality,Pan Quality
0,monkey_fetus_40_0272_eyes1.tif,0.7,0.739823,153,28,45,0.845304,0.772727,0.676991,0.807388,0.846171,0.683188
1,monkey_fetus_40_0273_bone.tif,0.7,0.729419,32,5,24,0.864865,0.571429,0.524590,0.688172,0.842054,0.579478
2,monkey_fetus_40_0281_eyes2.tif,0.7,0.682934,130,50,111,0.722222,0.539419,0.446735,0.617577,0.813083,0.502141
3,monkey_fetus_40_0308_liver1.tif,0.7,0.807160,135,13,20,0.912162,0.870968,0.803571,0.891089,0.863118,0.769115
4,monkey_fetus_40_0323_ears1.tif,0.7,0.670261,110,108,122,0.504587,0.474138,0.323529,0.488889,0.792253,0.387324
5,monkey_fetus_40_0344_brain1.tif,0.7,0.763148,196,40,87,0.830508,0.692580,0.606811,0.755299,0.845470,0.638583
6,monkey_fetus_40_0380_heart1.tif,0.7,0.758650,71,12,20,0.855422,0.780220,0.689320,0.816092,0.825936,0.674039
7,monkey_fetus_40_0381_thymus.tif,0.7,0.810057,134,11,35,0.924138,0.792899,0.744444,0.853503,0.852820,0.727884
8,monkey_fetus_40_0426_lungs.tif,0.7,0.692953,114,64,147,0.640449,0.436782,0.350769,0.519362,0.805344,0.418265
9,monkey_fetus_40_0428_thyroid1.tif,0.7,0.748314,98,13,24,0.882883,0.803279,0.725926,0.841202,0.844184,0.710129


In [15]:
results = []
for score in scores:
    results.append(score.score_set())

In [16]:
results

[                                   Image  Tau       IoU   TP   FP   FN  \
 0         monkey_fetus_40_0272_eyes1.tif  0.7  0.739823  153   28   45   
 1          monkey_fetus_40_0273_bone.tif  0.7  0.729419   32    5   24   
 2         monkey_fetus_40_0281_eyes2.tif  0.7  0.682934  130   50  111   
 3        monkey_fetus_40_0308_liver1.tif  0.7  0.807160  135   13   20   
 4         monkey_fetus_40_0323_ears1.tif  0.7  0.670261  110  108  122   
 5        monkey_fetus_40_0344_brain1.tif  0.7  0.763148  196   40   87   
 6        monkey_fetus_40_0380_heart1.tif  0.7  0.758650   71   12   20   
 7        monkey_fetus_40_0381_thymus.tif  0.7  0.810057  134   11   35   
 8         monkey_fetus_40_0426_lungs.tif  0.7  0.692953  114   64  147   
 9      monkey_fetus_40_0428_thyroid1.tif  0.7  0.748314   98   13   24   
 10       monkey_fetus_40_0451_heart2.tif  0.7  0.693792  105   50   82   
 11          monkey_fetus_40_0452_gi1.tif  0.7  0.647873   67   83  165   
 12       monkey_fetus_40

In [19]:
import pickle

# File path where you want to save the pickled object
file_path = r'C:\Users\Donald\PycharmProjects\monkey_nuc_segment\Segmentation_Analysis\monkey\thesis\pan_qual_dfs.pkl'

# Open the file in binary write mode and use pickle.dump() to serialize 'results'
with open(file_path, 'wb') as f:
    pickle.dump(results, f)